In [14]:
# TEST

input = """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
..........."""

In [128]:
# REAL

input = """Redacted"""

In [4]:
area_map = {}

start = (-1, -1)
max_i = -1
max_j = -1

for j, line in enumerate(input.splitlines()):
    max_j = max(max_j, j)
    for i, char in enumerate(line):
        max_i = max(max_i, i)
        area_map[(i,j)] = char
        if char == "S":
            start = (i,j)

visited = set()
visited.add(start)

for i in range(64):
    new_visited = set()
    for place in visited:
        for dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if (place[0] + dir[0], place[1] + dir[1]) in area_map and area_map[(place[0] + dir[0], place[1] + dir[1])] != "#":
                new_visited.add((place[0] + dir[0], place[1] + dir[1]))
    visited = new_visited

print("Part 1: ", len(visited))


3716


In [35]:
# Part 2 - Brute Force. Far too slow/memory intensive
area_map = {}

start = (-1, -1)
max_i = -1
max_j = -1

for j, line in enumerate(input.splitlines()):
    max_j = max(max_j, j)
    for i, char in enumerate(line):
        max_i = max(max_i, i)
        area_map[(i,j)] = char
        if char == "S":
            start = (i,j)

visited = set()
visited.add((start, (0,0)))

for i in range(500):
    new_visited = set()
    for place in visited:
        for dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_place = (((place[0][0] + dir[0]) % (max_i + 1), (place[0][1] + dir[1]) % (max_j + 1)), (place[1][0] + (1 if place[0][0] + dir[0] > max_i else -1 if place[0][0] + dir[0] < 0 else 0), place[1][1] + (1 if place[0][1] + dir[1] > max_j else -1 if place[0][1] + dir[1] < 0 else 0)))
            if area_map[new_place[0]] != "#":
                new_visited.add(new_place)
    visited = new_visited

print("Part 2: ", len(visited))

Part 2:  167004


In [3]:
def test_grid(mega, done, area_map):
    for k, v in area_map.items():
        if v != "#" and (mega not in done or v not in done[mega]):
            return False
    return True

In [41]:
# Part 2 - more optimised? Brute Force (still way way way too slow)
to_do = set()
to_do.add((start, (0,0)))
done = {}
done_grids = set()
even_count = 0
odd_count = 0

steps = 5000

for i in range(steps):
    new_to_do = set()
    for place in to_do:
        if place[1] not in done:
            done[place[1]] = set()
            done[place[1]].add(place[0])
            if i % 2 == 0:
                even_count += 1
            else:
                odd_count += 1
        else:
            if place[0] not in done[place[1]]:
                done[place[1]].add(place[0])
                if i % 2 == 0:
                    even_count += 1
                else:
                    odd_count += 1
        for dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_place = (((place[0][0] + dir[0]) % (max_i + 1), (place[0][1] + dir[1]) % (max_j + 1)), (place[1][0] + (1 if place[0][0] + dir[0] > max_i else -1 if place[0][0] + dir[0] < 0 else 0), place[1][1] + (1 if place[0][1] + dir[1] > max_j else -1 if place[0][1] + dir[1] < 0 else 0)))
            if area_map[new_place[0]] != "#":
                if new_place[1] in done_grids or (new_place[1] in done and new_place[0] in done[new_place[1]]):
                    continue
                new_to_do.add(new_place)
    if i % 500 == 0:
        # compact the grids
        to_remove = []
        for grid in done:
            if test_grid(grid, done, area_map):
                done_grids.add(grid)
                to_remove.append(grid)
        for grid in to_remove:
            done.pop(grid)
        # we can actually remove any "done grid" who's neighbours are all done because we'll never hit them again
        to_remove = []
        for grid in done_grids:
            nah = False
            for dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                if (grid[0] + dir[0], grid[1] + dir[1]) not in done_grids:
                    nah = True
            if not nah:
                to_remove.append(grid)
        for grid in to_remove:
            done_grids.pop(grid)

    to_do = new_to_do

print("Part 2: ", len(to_do) + (even_count if steps % 2 == 0 else odd_count))

Part 2:  16733044


In [88]:
area_map = {}

start = (-1, -1)
max_i = -1
max_j = -1

for j, line in enumerate(input.splitlines()):
    max_j = max(max_j, j)
    for i, char in enumerate(line):
        max_i = max(max_i, i)
        area_map[(i,j)] = char
        if char == "S":
            start = (i,j)

print(start)
print(max_i, max_j)

(65, 65)
130 130


In [27]:
# Reframe the problem
# actually want "all points who's shortest path (manhattan metric) is an odd/even number less than <steps>"
# Extremely suspicious property of the real input: straight clear lines NESW from the S point

distance_map = {}

to_do = set()
to_do.add(start)

steps = 5000
even = steps % 2 == 0

count = 0

i = -1

while len(to_do) > 0:
    i += 1
    if i > steps:
        break
    new_to_do = set()
    for place in to_do:
        if place not in distance_map:
            distance_map[place] = i
        for dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if area_map[((place[0] + dir[0]) % (max_i + 1), (place[1] + dir[1]) % (max_j + 1))] != "#" and (place[0] + dir[0], place[1] + dir[1]) not in distance_map:
                new_to_do.add((place[0] + dir[0], place[1] + dir[1]))
    to_do = new_to_do
    #compact distance map every 1000 or so cycles
    if i % 100 == 0:
        #before = len(distance_map)
        for k, v in distance_map.copy().items():
            if v < i - 1:
                distance_map.pop(k)
                if (v % 2 == 0) == even:
                    count += 1
        #print(before, len(distance_map))



for v in distance_map.values():
    if (v % 2 == 0) == even and v <= steps:
        count += 1

print("Part 2: ", count)

Part 2:  21955113


In [27]:
# Reframe the problem
# actually want "all points who's shortest path (manhattan metric) is an odd/even number less than <steps>"
# Extremely suspicious property of the real input: straight clear lines NESW from the S point


distance_maps = {}
starting_points = [(0, 65), (65, 0), (130, 65), (65, 130)]

to_check = [start,(0, 65), (65, 0), (130, 65), (65, 130),(0,0), (130,0), (0, 130), (130, 130)]

for starts in to_check:
    distance_map = {}
    to_do = set()
    to_do.add(starts)
    i = -1

    while len(to_do) > 0:
        i += 1
        new_to_do = set()
        for place in to_do:
            if place not in distance_map:
                distance_map[place] = i
            for dir in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                new_place = place[0] + dir[0], place[1] + dir[1]
                if new_place in area_map and area_map[new_place] != "#" and new_place not in distance_map:
                    new_to_do.add(new_place)
        to_do = new_to_do
    
    distance_maps[starts] = distance_map

for k,v in distance_maps.items():
    max_key = k
    max_val = -1
    for kk, vv in v.items():
        if vv > max_val:
            max_key = kk
            max_val = vv
    print(k, max_key, max_val)

count1 = 0 

for v in distance_maps[start].values():
    if v % 2 == 0 and v <= 64:
        count1 += 1

print("Part 1: ", count1)

count_even = 0
count_odd = 0

for v in distance_maps[start].values():
    if v % 2 == 0:
        count_even += 1
    if v % 2 == 1:
        count_odd += 1

print("Even Count:", count_even)
print("Odd Count:", count_odd)


(65, 65) (0, 0) 130
(0, 65) (130, 0) 195
(65, 0) (0, 130) 195
(130, 65) (0, 0) 195
(65, 130) (0, 0) 195
(0, 0) (130, 130) 260
(130, 0) (0, 130) 260
(0, 130) (130, 0) 260
(130, 130) (0, 0) 260
Part 1:  3716
Even Count: 7570
Odd Count: 7496


In [127]:
# How many grids can we (completely) hit in 26501365 steps?
# first take "grid co-ordinates" to start point plus step offset

def mega(grid):
    if grid[0] == 0:
        if grid[1] == 0:
            return (0, (0,0))
        elif grid[1] > 0:
            return (131 * (abs(grid[1]) - 1) + 66, (65, 130))
        else:
            return (131 * (abs(grid[1]) - 1) + 66, (65, 0)) 
    if grid[1] == 0:
        if grid[0] > 0:
            return (131 * (abs(grid[0]) - 1) + 66, (0, 65))
        else:
            return (131 * (abs(grid[0]) - 1) + 66, (130, 65)) 
        
    if grid[0] > 0 and grid[1] > 0:
        return (132 + (abs(grid[0]) - 1) * 131 + (abs(grid[1]) - 1)*131, (0, 130))
    if grid[0] < 0 and grid[1] > 0:
        return (132 + (abs(grid[0]) - 1) * 131 + (abs(grid[1]) - 1)*131, (130, 130))
    if grid[0] > 0 and grid[1] < 0:
        return (132 + (abs(grid[0]) - 1) * 131 + (abs(grid[1]) - 1)*131, (0, 0))
    if grid[0] < 0 and grid[1] < 0:
        return (132 + (abs(grid[0]) - 1) * 131 + (abs(grid[1]) - 1)*131, (130, 0))
    

num = 0
odd_num = 0
even_num = 0


# when i is 202299, middle grid is odd count
#81850175401
size = 202300
for i in range(size):
    num += 2*(1 + (size - 1 - i)*2)
    if i % 2 == 0:
#        total += 2*count_even*(1 + (3 - i) // 2) + 2* count_odd * (1 + (3 - i) // 2)
        odd_num += 2*(2*(1 + ((size - 1 - i) // 2)))
        even_num += 2*(1 + 2*((size - 1 - i) // 2))
    else:
        odd_num += 2* (1 + 2*((size - 1 - i) // 2))
        even_num += 2*(2*((size - 1 - i) // 2))

# double counted zero
num -= ((1 + (size - 1)*2))
odd_num -= 2*(1 + ((size - 1) // 2))
even_num -= (1 + 2*((size - 1) // 2))

# This appears to be the wrong way around. I have no idea exactly why and cba to convince myself of the logic but it gives the right answer.
total = count_even * odd_num + count_odd * even_num

print("Number of complete grids: ", num)
print("Odd grids: ", odd_num)
print("even grids: ", even_num)
assert(num == odd_num + even_num)
print("Number of points on complete grids: ", total)

big_map = {}

for i in range(1,202300):
    j = 202300 - i
    assert(i != 0 and j != 0)
    mega1 = mega((i, j))
    mega2 = mega((0 - i, j))
    mega3 = mega((i, 0 - j))
    mega4 = mega((0 - i, 0 - j))
    megas = [mega1, mega2, mega3, mega4]
    test = 0
    if tuple(megas) not in big_map:
        for test_mega in megas:
            specific_test = 0
            test_map = test_mega[1]
            test_offset = test_mega[0]
            for v in distance_maps[test_map].values():
                if (v + test_offset) <= 26501365 and (v + test_offset) % 2 == 1:
                    specific_test += 1
            assert(specific_test != 7570 and specific_test != 7496)
            test += specific_test
        big_map[tuple(megas)] = test
    else:
        test = big_map[tuple(megas)]
    total += test

# am I just wrong?    
for i in range(1,202301):
    j = 202301 - i
    assert(i != 0 and j != 0)
    grid1 = (i, j)
    grid2 = (0 - i, j)
    grid3 = (i, 0 - j)
    grid4 = (0 - i, 0 - j)
    grids = [grid1, grid2, grid3, grid4]
    # print(grids)
    megas = [mega(x) for x in grids]
    test = 0
    if tuple(megas) not in big_map:
        for test_mega in megas:
            specific_test = 0
            test_map = test_mega[1]
            test_offset = test_mega[0]
            for v in distance_maps[test_map].values():
                if (v + test_offset) <= 26501365 and (v + test_offset) % 2 == 1:
                    specific_test += 1
            assert(specific_test != 7570 and specific_test != 7496)
            test += specific_test
        big_map[tuple(megas)] = test
    else:
        test = big_map[tuple(megas)]
    total += test
# special case the corners
mega1 = mega((0, 202300))
mega2 = mega((0, -202300))
mega3 = mega((202300, 0))
mega4 = mega((-202300,0))
megas = [mega1, mega2, mega3, mega4]
test = 0
if tuple(megas) not in big_map:
    for test_mega in megas:
        test_map = test_mega[1]
        test_offset = test_mega[0]
        for v in distance_maps[test_map].values():
            specific_test = 0
            if (v + test_offset) <= 26501365 and (v + test_offset) % 2 == 1:
                specific_test += 1
            assert(specific_test != 7570 and specific_test != 7496)
            test += specific_test
    big_map[tuple(megas)] = test
else:
    test = big_map[tuple(megas)]
total += test

print("Part 2: ", total)

Number of complete grids:  81850175401
Odd grids:  40925290000
even grids:  40924885401
Number of points on complete grids:  616577386265896
Part 2:  616583483179597


In [125]:
test_mega = mega((202300, 1))
test = 0
test_map = test_mega[1]
test_offset = test_mega[0]
print(test_offset)
for v in distance_maps[test_map].values():
    if (v + test_offset) <= 26501365 and (v + test_offset) % 2 == 1:
        test += 1
print(test)

26501301
980


In [63]:
# Tried guesses:
616587242879999 - too high
616588051064504
616587242879999
616582673777371 - too low
616583453235418 - still wrong
616583453239271 - still wrong
616583483179597